In [85]:
import random
import numpy as np
import json

In [86]:
stage_01 = "Create a detailed persona for the intended user of a chair, focusing on their lifestyle, preferences, and values relevant to seating. Include key factors such as comfort needs, aesthetic tastes, and potential use environments. Additionally, create a visual portrait of this persona, capturing their characteristics and style preferences in a visualization."
stage_02 = "Define the main problem the chair design aims to solve, independently of any prior user insights or persona details. Outline functional and aesthetic requirements that the chair should meet, focusing on aspects like durability, comfort, and visual style. Create a mood board to capture these requirements, combining textures, colors, and style inspirations that align with the intended direction."
stage_03 = "Generate multiple design ideas for the chair without reference to prior user personas, problem definitions, or requirements. List each concept with descriptions of the chair’s form, shape, and material choices. Create an industrial design sketch that explores different forms and styles, visualizing the various possibilities for the chair design."
stage_04 = "Create a final design for the chair that is production-ready, focusing on specific materials, dimensions, and assembly instructions. Do not refer to any previous personas, problem definitions, or form ideas. Produce a high-quality rendering of the finalized design, showcasing materials, finishes, and structure in a realistic setting."
all_stages = [stage_01, stage_02, stage_03, stage_04]


text_prompts = ["Text Output: Persona description including user lifestyle, values, comfort needs, and aesthetic preferences.", "Text Output: Problem statement with a list of functional and aesthetic requirements for the chair.", "Text Output: List of design ideas, each with form descriptions and material choices.", "Text Output: Finalized design description with materials, dimensions, and assembly instructions."]
image_prompts = ["Image Output: Portrait of the persona that visually represents their style and characteristics.", "Image Output: Mood board that reflects the functional and aesthetic requirements.", "Image Output: Industrial design sketch showing various chair forms and styles.", "Image Output: High-resolution rendering of the final chair design in a realistic setting."]

In [87]:
def extra_emotion(valence, arousal):
  emotion = f"""
  Take into account my current emotional state without explicitly mentioning it, scales ranging from -1 to 1.
  Arousal: {arousal}
  Valence: {valence}
  """
  return emotion

In [88]:
def make_prompts(stage_num, all_scores, stage):
    current_stage = all_scores[f'stage_{stage_num}']['prompts']
    pos = all_scores['positive']
    neg = all_scores['negative']

    prompt_baseline = stage
    prompt_pos_h_arousal= stage + " " + extra_emotion(pos['valence'], pos['arousal_high'])
    prompt_neg_h_arousal = stage + " " + extra_emotion(neg['valence'], neg['arousal_high'])

    prompt_pos_l_arousal= stage + " " + extra_emotion(pos['valence'], pos['arousal_low'])
    prompt_neg_l_arousal = stage + " " + extra_emotion(neg['valence'], neg['arousal_low'])

    # text
    current_text = current_stage['text_prompt']
    current_text['B'] = prompt_baseline + text_prompts[stage_num-1]
    current_text['PHA'] = prompt_pos_h_arousal + text_prompts[stage_num-1]
    current_text['NHA'] = prompt_neg_h_arousal + text_prompts[stage_num-1]
    current_text['PLA'] = prompt_pos_l_arousal + text_prompts[stage_num-1]
    current_text['NLA'] = prompt_neg_l_arousal + text_prompts[stage_num-1]

    # images
    current_image = current_stage['image_prompt']
    current_image['B'] = prompt_baseline + image_prompts[stage_num-1]
    current_image['PHA'] = prompt_pos_h_arousal + image_prompts[stage_num-1]
    current_image['NHA'] = prompt_neg_h_arousal + image_prompts[stage_num-1]
    current_image['PLA'] = prompt_pos_l_arousal + image_prompts[stage_num-1]
    current_image['NLA'] = prompt_neg_l_arousal + image_prompts[stage_num-1]

    # all without specifics
    current_text = current_stage['all']
    current_text['B'] = prompt_baseline 
    current_text['PHA'] = prompt_pos_h_arousal 
    current_text['NHA'] = prompt_neg_h_arousal 
    current_text['PLA'] = prompt_pos_l_arousal 
    current_text['NLA'] = prompt_neg_l_arousal 
    

In [89]:
all_scores = {'positive': {'valence': 0.5, 'arousal_high': 0.5, 'arousal_low': -0.5},
              'negative': {'valence': -0.5, 'arousal_high': 0.5, 'arousal_low': -0.5},
              'stages': len(all_stages)}


for i, stage in enumerate(all_stages):
  i += 1
  all_scores[f'stage_{i}'] = {'prompts': {'text_prompt': {}, 'image_prompt': {}, 'all': {}}, 'output_text': {}, 'sentiment': {}}
  make_prompts(i, all_scores, stage)

with open('all_scores.json', 'w') as f:
  f.write(json.dumps(all_scores, indent=3))
